In [1]:
%load_ext autoreload
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np

import datetime

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV


import feature_transformers as ft
from kaggle_helper import KaggleHelper
%aimport feature_transformers
%aimport kaggle_helper

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
%aimport

Modules to reload:
feature_transformers kaggle_helper

Modules to skip:



In [3]:
kh = KaggleHelper("pipeline testing.db")

table metrics already exists


In [50]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
y = raw[['TripType', 'VisitNumber']].groupby('VisitNumber').mean()
X = raw.drop('TripType', axis=1)

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


In [51]:
%autoreload

In [52]:
print(kh.current_run)
kh.start_pipeline()
print(kh.record_metric("validation", "start", "", "", ""))
print(kh.current_run)

1448485202
None
1448485207


In [88]:
%autoreload
y = raw[['TripType', 'VisitNumber']].groupby('VisitNumber').mean()
X = raw.drop('TripType', axis=1)

dummy_cols = ['Weekday']
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']

funcs = [np.sum, np.count_nonzero]
n_components = [10, 15]
Cs = np.logspace(-4, 4, 3)
cv_grid = {'logistic__C': Cs, 'pca__n_components': n_components}
num_folds = 6


logistic = linear_model.LogisticRegression()
pca = decomposition.PCA()
dfta = ft.DataFrameToArray()
add_returns = ft.NGAddReturns()
gdd = ft.GDummyAndKeepTransform(dummy_cols, keep_cols, funcs)

kh.start_pipeline()
transform_steps =  list(ft.wrapStep(kh, ("add_returns", add_returns)) + ft.wrapStep(kh, ("gdd", gdd)))
transform_steps.append((("dfta",dfta)))
transform_pipe = Pipeline(steps=transform_steps)
X = transform_pipe.transform(X)

pred_steps = [('pca', pca)]
pred_steps.append(('logistic', logistic))

pred_pipe = Pipeline(steps=pred_steps)

In [90]:
estimator = GridSearchCV(pred_pipe, cv_grid)

In [ ]:
X.shape

(95674, 20)

In [ ]:
estimator.fit(X, y['TripType'].values)